In [8]:
# 详解Python中的__init__和__new__
## 一、__init__ 方法是什么？
"""使用Python写过面向对象的代码的同学，可能对 __init__ 方法已经非常熟悉了，__init__ 方法通常用在初始化一个类实例的时候。例如："""
class Person(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __str__(self):
        return '<Person: %s(%s)>' % (self.name, self.age)
piglei = Person('piglei', 24)
print(piglei)

<Person: piglei(24)>


In [18]:
"""这样便是__init__最普通的用法了。但__init__其实不是实例化一个类的时候第一个被调用 的方法。当使用 Persion(name, age) 这样的表达式来实例化一个类时，最先被调用的方法 其实是 __new__ 方法。"""
## 二、__new__ 方法是什么？
"""__new__方法接受的参数虽然也是和__init__一样，但__init__是在类实例创建之后调用，而 __new__方法正是创建这个类实例的方法。"""
"""In Python 3.3 and later, if you're overriding both __new__ and __init__, you need to avoid passing any extra arguments to the object methods you're overriding. If you only override one of those methods, it's allowed to pass extra arguments to the other one (since that usually happens without your help).

So, to fix your class, change the __new__ method like so:

def __new__(cls, nom, prenom):
    print("Appel de la méthode __new__ de la classe {}".format(cls))
    return object.__new__(cls) # don't pass extra arguments here!"""
class Person(object):
    """Silly Person"""
 
    def __new__(cls, name, age):
        print('__new__ called.')
        return super(Person, cls).__new__(cls)
 
    def __init__(self, name, age):
        print('__init__ called.')
        self.name = name
        self.age = age
 
    def __str__(self):
        return '<Person: %s(%s)>' % (self.name, self.age)
piglei = Person('piglei', 24)
print(piglei)

__new__ called.
__init__ called.
<Person: piglei(24)>


In [43]:
"""通过运行这段代码，我们可以看到，__new__方法的调用是发生在__init__之前的。其实当 你实例化一个类的时候，具体的执行逻辑是这样的：

1.p = Person(name, age)
2.首先执行使用name和age参数来执行Person类的__new__方法，这个__new__方法会 返回Person类的一个实例（通常情况下是使用 super(Persion, cls).__new__(cls, … …) 这样的方式），
3.然后利用这个实例来调用类的__init__方法，上一步里面__new__产生的实例也就是 __init__里面的的 self
所以，__init__ 和 __new__ 最主要的区别在于：
1.__init__ 通常用于初始化一个新实例，控制这个初始化的过程，比如添加一些属性， 做一些额外的操作，发生在类实例被创建完以后。它是实例级别的方法。
2.__new__ 通常用于控制生成一个新实例的过程。它是类级别的方法。
但是说了这么多，__new__最通常的用法是什么呢，我们什么时候需要__new__？"""
## 三、__new__ 的作用
"""依照Python官方文档的说法，__new__方法主要是当你继承一些不可变的class时(比如int, str, tuple)， 提供给你一个自定义这些类的实例化过程的途径。还有就是实现自定义的metaclass。
首先我们来看一下第一个功能，具体我们可以用int来作为一个例子：
假如我们需要一个永远都是正数的整数类型，通过集成int，我们可能会写出这样的代码。"""
class PositiveInteger(int):
    def __init__(self, value):
        return super(PositiveInteger, self).__init__(self, abs(value))
        
    def __new__(cls, *args, **kwargs):
        super().__new__(cls)
    
i = PositiveInteger(-3)
print(i)

None


In [44]:
class PositiveInteger(int):
    def __new__(cls, value):
        return super().__new__(cls, abs(value))

i = PositiveInteger(-3)
print(i)

3


In [27]:
abs(-3)

3

In [41]:
"""通过重载__new__方法，我们实现了需要的功能。
另外一个作用，关于自定义metaclass。其实我最早接触__new__的时候，就是因为需要自定义 metaclass，但鉴于篇幅原因，我们下次再来讲python中的metaclass和__new__的关系。"""
## 四、用__new__来实现单例
"""事实上，当我们理解了__new__方法后，我们还可以利用它来做一些其他有趣的事情，比如实现 设计模式中的 单例模式(singleton) 。
因为类每一次实例化后产生的过程都是通过__new__来控制的，所以通过重载__new__方法，我们 可以很简单的实现单例模式。"""
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance =  super().__new__(cls)
        return cls.instance
obj1 = Singleton()
obj2 = Singleton()
obj1.attr1 = 'value1'
print(obj1.attr1, obj2.attr1)

value1 value1


In [42]:
print(obj1 is obj2)

True
